# TME1 : premier pas en traitement d'image avec Python et Jupyter

> Consignes: le fichier **TME1_Sujet.ipynb** est à déposer sur le site Moodle de l'UE https://moodle-sciences.upmc.fr/moodle-2018/course/view.php?id=4650. Si vous êtes en binôme, renommez-le en **TME1_nom1_nom2.ipynb**.

Pour ce premier TME, qui consiste à réaliser des opérations de base de traitement d'image, nous utiliserons uniquement le language Python natif, sans module particulier et cela pour bien comprendre ce qu'est une image sur le disque et sa représentation en mémoire, une liste de valeurs, organisée en lignes de pixels.

Les deux fonctions suivantes permettent respectivement de lire et d'écrire sur le disque des images au format Portable Grey Map (PGM). Elle utilise le type formel <tt>Image</tt> qui est <tt>tuple[list[int],int,int]</tt>, selon les conventions du cours 1I001. Le premier élément est la liste des pixels de l'image, ordonnée ligne par ligne, le second élément est la longueur d'une ligne (soit le nombre de pixels dans une ligne), et le troisième le nombre de lignes.


In [1]:
def readPGM(file):
    """  str -> tuple[list[int],int,int] | NoneType
    Lit un fichier PGM et retourne la liste des valeurs de 
    l'image et ses dimensions (nombre de colonnes, nombre de lignes).
    """
    fp = open(file,'rb')
    # en mode binaire, readline() retourne un type 'bytes'
    if fp.readline() == b'P5\n':
        while True:
            # lecture d'une ligne, conversion vers str
            line = fp.readline().decode()
            if line[0] != '#': break
        # découpage en mots, puis conversion        
        w,h=line.split()
        w,h=int(w),int(h)
        # Nb de niveaux de gris (pas utile mais il faut le lire)
        l=fp.readline()
        # données
        data = list(fp.read(w*h))
        if len(data) != w*h:
            print ('readPGM: error with ' + file + ': has wrong size')
        fp.close()
        return (data,w,h)
    else:
        print( 'readPGM: error with '+ file + ': unsupported format')
    fp.close()
    return None

def writePGM(image,file):
   """ tuple(list[int],int,int)*str -> NoneType
   Ecrit une image au format PGM 
   """
   data,w,h = image
   fp = open(file,'wb')
   fp.writelines([bytes('P5\n'+str(w)+' '+str(h)+'\n255\n','utf8')])
   fp.write(bytearray(data))
   fp.close()

Pour voir une image, nous utiliserons une commande de votre système (normalement Linux Debian) qui affichera l'image lu depuis le disque. Si vous utilisez un autre système d'exploitation (OSX ou Windows), il faudra utiliser un programme spécifique.     

In [2]:
from os import system

def viewImage(image):
    """ tuple[list[int],int,int] -> NoneType
    Lit et affiche une image depuis le disque
    """
    writePGM(image,'/tmp/viewimage.pgm')
    system( 'xdg-open /tmp/viewimage.pgm')
    #system( 'rm -f /tmp/viewimage.pgm')


# Exercice 1: visualisation et histogramme

## 1.1 Voir une image
Écrire un code python qui lit l'image <tt>figs/carrefou.pgm</tt> l'affiche à l'écran et imprime ses dimensions.

In [9]:
p=readPGM("img/carrefou.pgm")
print(p)
#viewImage(p)
print(p[1], p[2])

([38, 96, 77, 77, 74, 82, 83, 78, 77, 79, 88, 83, 77, 86, 84, 79, 83, 80, 81, 80, 77, 72, 67, 64, 67, 68, 72, 72, 47, 88, 96, 62, 79, 86, 122, 121, 103, 125, 107, 87, 98, 85, 63, 53, 47, 63, 75, 65, 63, 66, 64, 62, 64, 64, 66, 66, 70, 64, 63, 68, 82, 76, 64, 77, 76, 70, 73, 67, 74, 69, 61, 73, 72, 69, 70, 66, 59, 61, 73, 59, 51, 66, 74, 61, 59, 63, 62, 61, 68, 64, 51, 59, 67, 68, 67, 61, 59, 58, 56, 59, 62, 54, 47, 49, 68, 76, 61, 62, 70, 62, 65, 68, 61, 70, 66, 65, 73, 62, 64, 69, 66, 71, 66, 59, 66, 69, 65, 66, 67, 62, 67, 94, 95, 66, 61, 76, 79, 64, 71, 58, 70, 89, 68, 66, 54, 60, 71, 72, 88, 87, 87, 83, 87, 96, 72, 87, 85, 73, 78, 65, 85, 92, 74, 82, 81, 78, 80, 75, 68, 71, 71, 60, 60, 70, 64, 64, 69, 73, 70, 56, 53, 72, 71, 62, 67, 64, 56, 61, 71, 64, 66, 69, 64, 65, 79, 88, 93, 92, 95, 107, 106, 110, 108, 76, 69, 77, 79, 80, 73, 67, 68, 72, 70, 70, 81, 74, 70, 89, 90, 81, 83, 88, 86, 77, 75, 98, 117, 122, 105, 75, 65, 73, 91, 97, 84, 95, 94, 81, 75, 76, 81, 71, 73, 81, 72, 74, 72

## 1.2 Calcul d'histogramme

Écrire une fonction <tt>histogram()</tt> qui prend la liste des valeurs d'une image, un entier <tt>n</tt> et retourne une liste des <tt>n</tt> valeurs représentant l'histogramme de l'image.

In [12]:
def histogram(data,n):
    """ list[int]*int > list[int] """
    H=[0]*(n+1)
    for v in data:
        H[v]+=1
    return H

#p=readPGM("img/carrefou.pgm")
print(histogram(p[0], 255))
q=histogram(p[0], 255)

[0, 0, 2, 0, 0, 0, 2, 4, 8, 9, 10, 5, 14, 19, 63, 370, 663, 318, 123, 135, 145, 167, 197, 184, 238, 285, 291, 335, 382, 347, 411, 422, 445, 518, 544, 564, 598, 646, 709, 686, 795, 852, 960, 1012, 1084, 1221, 1255, 1277, 1379, 1416, 1540, 1606, 1693, 1689, 1746, 1794, 1748, 1702, 1702, 1673, 1578, 1521, 1388, 1378, 1270, 1207, 1128, 1067, 1035, 1007, 930, 920, 875, 789, 740, 731, 651, 573, 608, 563, 495, 558, 467, 418, 405, 351, 346, 345, 310, 285, 278, 263, 222, 221, 215, 201, 209, 188, 180, 153, 144, 160, 135, 104, 119, 96, 86, 73, 73, 56, 67, 49, 36, 41, 24, 37, 22, 22, 23, 17, 6, 14, 24, 9, 7, 6, 4, 10, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


## 1.3 Affichage d'histogramme
Écrire un code Python qui charge une image, calcule son histogramme, et l'affiche l'histogramme sous forme textuelle, par exemple:
<pre>
0 -> 3
1 -> 5
2 -> 10
 ...
</pre>
Dans cet exemple, il faut lire qu'il y a 3 pixels de valeur 0, 5 pixels de valeur 1 et 10 pixels de valeur 2, _etc_.


In [13]:
image= readPGM("img/lena.pgm")
q=histogram(image[0],255)
for i in range (len(q)):
    print (i, "->", q[i])

0 -> 0
1 -> 0
2 -> 0
3 -> 0
4 -> 0
5 -> 0
6 -> 0
7 -> 0
8 -> 0
9 -> 0
10 -> 0
11 -> 0
12 -> 0
13 -> 0
14 -> 0
15 -> 0
16 -> 0
17 -> 0
18 -> 0
19 -> 0
20 -> 2
21 -> 1
22 -> 11
23 -> 32
24 -> 59
25 -> 82
26 -> 118
27 -> 179
28 -> 251
29 -> 302
30 -> 356
31 -> 449
32 -> 539
33 -> 716
34 -> 837
35 -> 968
36 -> 1085
37 -> 1342
38 -> 1571
39 -> 1734
40 -> 2001
41 -> 2104
42 -> 2272
43 -> 2317
44 -> 2214
45 -> 2041
46 -> 1923
47 -> 1896
48 -> 1865
49 -> 1796
50 -> 1684
51 -> 1616
52 -> 1465
53 -> 1258
54 -> 1185
55 -> 1109
56 -> 1104
57 -> 1056
58 -> 954
59 -> 918
60 -> 753
61 -> 728
62 -> 702
63 -> 745
64 -> 812
65 -> 927
66 -> 905
67 -> 909
68 -> 857
69 -> 839
70 -> 834
71 -> 882
72 -> 859
73 -> 909
74 -> 813
75 -> 855
76 -> 783
77 -> 813
78 -> 854
79 -> 1001
80 -> 1023
81 -> 1008
82 -> 990
83 -> 999
84 -> 1032
85 -> 1093
86 -> 1191
87 -> 1205
88 -> 1374
89 -> 1477
90 -> 1596
91 -> 1524
92 -> 1544
93 -> 1619
94 -> 1677
95 -> 1821
96 -> 1800
97 -> 1861
98 -> 1953
99 -> 1909
100 -> 1822
101 -

# Exercice 2: étirement d'histogramme
## 2.1 Étirement 
Écrire la fonction <tt>etire()</tt> qui prend un histogramme et retourne un tableau (une liste Python) des correspondance entre niveaux de gris avant et après étirement. L'histogramme doit être étiré sur la dynamique la plus grande, c'est-à-dire [0,255]. Si <tt>table</tt> est la liste retournée et si <tt>table[0]</tt> vaut 2, cela signifie que les pixels ayant le niveau de gris 0 auront alors la valeur 2 après étirement de l'histogramme. Indication: utiliser une des fonctions d'arrondis (<tt>floor</tt>, <tt>ceil</tt>, ...) du module Python <tt>math</tt>

In [17]:
from math import ceil

def etire(his):
    """ list[int] -> list[int] """
    deb=0
    fin=0
    i=0
    j=len(his)-1
    while(his[i]==0):
        i+=1
        
    deb=i
    while(his[j]==0):
        j-=1
        
    fin=j
    alpha= 255/(fin-deb)
    
    I2=[0]*len(his)
    
    for r in range (deb, fin+1):
        I2[r]=ceil(alpha*(r-deb))
    
    return I2

#image= readPGM("img/lena.pgm")
#q=histogram(image[0],255)
e=etire(q)
print(e)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 3, 4, 5, 7, 8, 9, 10, 12, 13, 14, 15, 17, 18, 19, 20, 21, 23, 24, 25, 26, 28, 29, 30, 31, 33, 34, 35, 36, 37, 39, 40, 41, 42, 44, 45, 46, 47, 49, 50, 51, 52, 53, 55, 56, 57, 58, 60, 61, 62, 63, 65, 66, 67, 68, 69, 71, 72, 73, 74, 76, 77, 78, 79, 81, 82, 83, 84, 85, 87, 88, 89, 90, 92, 93, 94, 95, 97, 98, 99, 100, 102, 103, 104, 105, 106, 108, 109, 110, 111, 113, 114, 115, 116, 118, 119, 120, 121, 122, 124, 125, 126, 127, 129, 130, 131, 132, 134, 135, 136, 137, 138, 140, 141, 142, 143, 145, 146, 147, 148, 150, 151, 152, 153, 154, 156, 157, 158, 159, 161, 162, 163, 164, 166, 167, 168, 169, 170, 172, 173, 174, 175, 177, 178, 179, 180, 182, 183, 184, 185, 187, 188, 189, 190, 191, 193, 194, 195, 196, 198, 199, 200, 201, 203, 204, 205, 206, 207, 209, 210, 211, 212, 214, 215, 216, 217, 219, 220, 221, 222, 223, 225, 226, 227, 228, 230, 231, 232, 233, 235, 236, 237, 238, 239, 241, 242, 243, 244, 246, 247, 248, 249, 251, 252, 253

## 2.2 Changement des valeurs de l'image
Écrire la fonction <tt>applique()</tt> qui applique la table obtenue par la fonction <tt>etire()</tt> à la liste des valeurs de l'image pour former une nouvelle image son histogramme étiré.

In [18]:
def applique(data,table):
    """ list[int]*list[int] -> list[int] """
    for i in range (len(data)):
        data[i]= table[data[i]]
    return data

## 2.3 Application
Mettre en oeuvre les fonctions précédentes sur les images mises à votre disposition. Visualisez le résultat à l'aide de la fonction <tt>viewImage()</tt>.
Attention: certaines images ont déjà un histogramme étiré ! Comment le vérifier ? (répondre en commentaire dans le code Python ci-dessous).

In [24]:
#carrefou
p=readPGM("img/carrefou.pgm")
data, w, h =p
p1=histogram(data, w*h)
table=etire(p1)
print(applique(data, table))

writePGM(p, "carrefou_eti.pgm")
system('xdg-open carrefou_eti.pgm')


## certaines images sont déjà étirées car ...

In [ ]:
#angiogra
p=readPGM("img/angiogra.pgm")
data, w, h =p
p1=histogram(data, w*h)
table=etire(p1)
print(applique(data, table))

writePGM(p, "angio_eti.pgm")
system('xdg-open angio_eti.pgm')

In [ ]:
#belemn

p=readPGM("img/belemn.pgm")
data, w, h =p
p1=histogram(data, w*h)
table=etire(p1)
print(applique(data, table))

writePGM(p, "belemn_eti.pgm")
system('xdg-open belemn.pgm')


In [ ]:
#couloir


p=readPGM("img/couloir.pgm")
data, w, h =p
p1=histogram(data, w*h)
table=etire(p1)
print(applique(data, table))

writePGM(p, "couloir_eti.pgm")
system('xdg-open couloir.pgm')


In [ ]:
#echograp

p=readPGM("img/echograp.pgm")
data, w, h =p
p1=histogram(data, w*h)
table=etire(p1)
print(applique(data, table))

writePGM(p, "echo_eti.pgm")
system('xdg-open echo_eti.pgm')


In [ ]:
#lena

p=readPGM("img/lena.pgm")
data, w, h =p
p1=histogram(data, w*h)
table=etire(p1)
print(applique(data, table))

writePGM(p, "lena_eti.pgm")
system('xdg-open lena_eti.pgm')


In [ ]:
#muscle

p=readPGM("img/muscle.pgm")
data, w, h =p
p1=histogram(data, w*h)
table=etire(p1)
print(applique(data, table))

writePGM(p, "muscle_eti.pgm")
system('xdg-open muscle_eti.pgm')



In [ ]:
#oursin

p=readPGM("img/oursin.pgm")
data, w, h =p
p1=histogram(data, w*h)
table=etire(p1)
print(applique(data, table))

writePGM(p, "oursin_eti.pgm")
system('xdg-open oursin_eti.pgm')

L'image couloir est déjà étirée car pour tout table[i]=i


# Exercice 3: seuillage d'images
## 3.1 Fonction de seuillage
Écrire la fonction <tt>seuillage()</tt> qui prend une liste de pixels <tt>data</tt>, un paramètre de seuil <tt>t</tt>. La liste de valeurs retournées sera consitituée d'une liste de valeur à 0 (pour les valeurs en dessous du seuil <tt>t</tt>) ou 255 (pour les autres valeurs).


In [25]:
def seuillage(data,t):
    """ list[int]*int -> list[int] """
    for i in range (len(data)):
        if (data[i]<t):
            data[i]=0
        else :
            data[i]=255
    return data

## 3.2 Application
Appliquer la fonction de seuillage à différentes images et faire varier le seuil. Utiliser la fonction <tt>viewImage()</tt> pour visualiser ces opérations de seuillage.

In [26]:
import random

p=readPGM("img/carrefou.pgm")
data, w, h =p
q=seuillage(data, random.randint(0, 255))
print(q)


[255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,

In [27]:
p=readPGM("img/oursin.pgm")
data, w, h =p
q=seuillage(data, random.randint(0, 255))
print(q)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

# Exercice 4: égalisation d'histogramme 
## 4.1 Egalisation
Écrire la fonction <tt>egalisation()</tt> qui prend un histogramme, applique une égalisation d'histogramme et retourne la table des correspondances entre niveaux de gris avant et après égalisation. On rappelle la formule d'égalisation vue en cours: $k' = Int\left(\frac{L-1}{N \times M}H_c(k)\right)$ où $H_c$ est l'histogramme cumulé, $N\times M$ la taille de l'image et $L$ la dynamique de l'image.

In [28]:
def egalisation(his):
    """ list[int] -> list[int] """
    #création de l'histogramme cumulé
    Hc = [0]*256
    for i in range(1, 256):
        Hc[i] = Hc[i-1] + his[i]
        
    #égalisation
    table = [0]*256
    for i in range(256):
        table[i] = int((255/(w*h))*Hc[i])
    print("table",table)
    return table

def applique_egal(data,table):
    """ list[int]*list[int] -> list[int] """
    for i in range(len(data)):
        data[i] = table[data[i]]
    return data
    
    

## 4.2 Application
Écrire un code Python qui lit une image, réalise son égalisation d'histogramme et affiche la nouvelle image égalisée.

In [29]:
# carrefou.pgm
imagehist = readPGM("img/carrefou.pgm")
data,w,h = imagehist
H = histogram(data, w*h)

table = egalisation(H)
applique(data, table)

writePGM(imagehist,'carrefou_egal.pgm')
system('xdg-open carrefou_egal.pgm')

# lena.pgm
imagehist = readPGM("img/lena.pgm")
data,w,h = imagehist
H = histogram(data, w*h)

table = egalisation(H)
applique(data, table)

writePGM(imagehist,'lena_egal.pgm')
system('xdg-open lena_egal.pgm')

# couloir.pgm
imagehist = readPGM("img/couloir.pgm")
data,w,h = imagehist
H = histogram(data, w*h)

table = egalisation(H)
applique(data, table)

writePGM(imagehist,'couloir_egal.pgm')
system('xdg-open couloir_egal.pgm')

table [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 4, 5, 6, 6, 7, 8, 8, 9, 10, 11, 12, 13, 15, 16, 18, 20, 21, 23, 25, 28, 30, 32, 35, 38, 41, 44, 48, 52, 56, 61, 66, 71, 76, 82, 88, 94, 100, 107, 114, 121, 128, 134, 141, 147, 154, 159, 165, 170, 175, 180, 184, 188, 192, 196, 200, 204, 207, 210, 213, 216, 218, 220, 223, 225, 227, 229, 231, 233, 234, 236, 237, 238, 239, 241, 242, 243, 243, 244, 245, 246, 247, 248, 248, 249, 249, 250, 251, 251, 251, 252, 252, 252, 253, 253, 253, 253, 253, 254, 254, 254, 254, 254, 254, 254, 254, 254, 254, 254, 254, 254, 254, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,

1